# Aprendizado por reforço

# Busca Não Informada: Quebrar Senhas.


**Objetivo:** Encontrar uma senha gerada aleatoriamente, com um número desconhecido de dígitos. Explorar como os algoritmos de busca não informada, incluindo a Busca em Profundidade (DFS), Busca em Largura (BFS) e Busca Gulosa, podem ser aplicados para resolver esse quebra-cabeça de senha.

- **$m$** (Profundidade Máxima): Isso representa o comprimento máximo de qualquer caminho no espaço de estados.
- **$b$** (Fator de Ramificação): Refere-se ao número máximo de sucessores que qualquer nó pode ter.
- **$d$** (Profundidade do Objetivo): Indica o número de etapas do caminho da raiz até o estado objetivo mais próximo.
Nomeclatura usada no relatório:

Relação esperada para cada algoritmo:

|       Método       |       Completeza       |     Otimização     |     Complexidade de Tempo     |     Complexidade de Espaço     |
|:------------------:|:----------------------:|:-------------------:|:-----------------------------:|:-----------------------------:|
|        BFS         |  Garante a resposta   |  Não é necessariamente ótimo, mas neste caso sim. |  $O(b^d)$                      |  $O(b^d)$                      |
|        DFS         |  Garante a resposta   |  Não necessariamente, mas neste caso sim. |  $O(b^m)$                      |  $O(b \cdot m)$                |
|   Busca Gulosa     |  Não garante a resposta, mas neste caso sim.  |  Não necessariamente, mas neste caso sim.  |  -              |  -              |



**Espaço de Estados**: O espaço de estados desse problema é composto por todas as possíveis combinações de dígitos que podem formar a senha. Suponhamos que cada dígito da senha possa variar de 0 a 9, e que a senha tenha um número máximo de dígitos igual a $m$. Portanto, o espaço de estados consiste em todas as sequências possíveis de dígitos de comprimento variável, de 1 a $m$ dígitos.

Por exemplo, se $m$ for igual a 4, o espaço de estados incluirá todas as sequências possíveis de dígitos, desde "0" até "9999", incluindo todas as combinações intermediárias, como "123", "4567" e assim por diante.

**Funções de Transição:**

As funções de transição determinam como podemos navegar no espaço de estados a partir de um estado atual. Para esse problema, as funções de transição são simples:

DFS (Busca em Profundidade): A função de transição na Busca em Profundidade consiste em adicionar um dígito ao estado atual e explorar todas as possíveis combinações para o próximo dígito, seguindo o caminho em profundidade até encontrar a senha correta ou atingir a profundidade máxima $m$.

BFS (Busca em Largura): A função de transição na Busca em Largura consiste em adicionar um dígito ao estado atual e explorar todas as possíveis combinações para o próximo dígito, seguindo o caminho em largura, ou seja, explorando primeiro todas as combinações de 1 dígito, depois todas as combinações de 2 dígitos e assim por diante.

Busca Gulosa: A Busca Gulosa não possui uma função de transição específica como as outras abordagens, pois não explora todas as possíveis combinações. Em vez disso, a Busca Gulosa se baseia na similaridade da senha tentada com a original antes de prosseguir para o próximo nó.


### Biblioteca:


In [ ]:
%%writefile search_pass.c
#include "search_pass.h"
#include <stdio.h>
#include <string.h>
#include <stdlib.h>

unsigned long long int iteracoes = 0;
unsigned long long int no_alocados = 0;

void gen_senha(char *senha, int tamanho, int min, int max) {

    if (min < MIN_ASCII_NUM) min = MIN_ASCII_NUM;
    if (max > MAX_ASCII_NUM) max = MAX_ASCII_NUM;

    for (int i = 0; i < TAMANHO_STRING; i++) {
        int num = rand() % (max - min + 1) + min;
        senha[i] = (char)num;
    }
    senha[TAMANHO_STRING] = '\0';
}

int calcularCusto(char *senhaAlvo, int tamanho, char *tentativa) {
    int custo = 0;
    for (int i = 0; i < tamanho; i++) {
      if(senhaAlvo[i] == tentativa[i]){
        custo++;
      }
    }
    return custo;
}

void reset_senha(char* senha, int value){
    int cont = 0;
    while(cont < TAMANHO_STRING){
        senha[cont] = (char)value;
        cont++;
    }
    senha[TAMANHO_STRING] = '\0';
}

int gulosa_senha(char *senhaAlvo, char *melhorTentativa, int min, int max) {
    int custoAtual, melhorCusto = 0;
    char melhorCen;
    int tamanho = strlen(senhaAlvo);
    iteracoes = 0;
    no_alocados = 0;

    for (int i = 0; i < tamanho; i++) {
        melhorCusto = 0;
        iteracoes++;
        no_alocados = i;
        for (int num = min; num <= max; num++) {
            melhorTentativa[i] = (char)num;
            custoAtual = calcularCusto(senhaAlvo, i + 1, melhorTentativa);
            iteracoes++;

            if (custoAtual > melhorCusto) {
                melhorCusto = custoAtual;
                melhorCen = (char)num;
            }
        }
        melhorTentativa[i] = melhorCen;
    }
    senhaAlvo[TAMANHO_STRING] = '\0';
    melhorTentativa[TAMANHO_STRING] = '\0';
    return (melhorCusto == tamanho);
}

int senha_DFS(char *senhaAlvo, char *tentativa, int index, int min, int max) {
    iteracoes++;
    no_alocados++;

    if (index == TAMANHO_STRING) {
        if (strcmp(senhaAlvo, tentativa) == 0) {
            return 1;
        }
        no_alocados--;
        return 0;
    }

    for (int num = min; num <= max; num++) {
        tentativa[index] = (char)num;
        if (senha_DFS(senhaAlvo, tentativa, index + 1, min, max)) {
            return 1;
        }
    }

    no_alocados--;
    return 0;
}


int senha_BFS(char *senhaAlvo, char *tentativa, int min, int max) {
    int level = 0;
    int i, j;

    while (level < TAMANHO_STRING) {
        if (strcmp(senhaAlvo, tentativa) == 0) {
            return 1;
        }

        tentativa[level]++;
        if (tentativa[level] > max) {
            tentativa[level] = (char)min;
            for (j = level - 1; j >= 0; j--) {
                tentativa[j]++;
                if (tentativa[j] <= max) break;
                tentativa[j] = (char)min;
            }
            if (j < 0) {
                break;
            }
        }

        if (level == TAMANHO_STRING - 1) {
            for (i = level; i >= 0; i--) {
                if (tentativa[i] < max) break;
                tentativa[i] = (char)min;
            }
            level = i;
        } else {
            for (i = level + 1; i < TAMANHO_STRING; i++) {
                tentativa[i] = (char)min;
            }
            level = TAMANHO_STRING - 1;
        }
    }

    return 0;
}

Overwriting search_pass.c


### Cabeçalho:


In [ ]:
%%writefile search_pass.h
#ifndef SEARCH_PASS_H
#define SEARCH_PASS_H

#define TAMANHO_STRING 10
#define MIN_ASCII_NUM 33
#define MAX_ASCII_NUM 126

extern unsigned long long int iteracoes;
extern unsigned long long int no_alocados;

void gen_senha(char *senha, int tamanho, int min, int max);
int gulosa_senha(char *senhaAlvo, char *tentativa, int min, int max);
int calcularCusto(char *senhaAlvo, int tamanho, char *tentativa);
int senha_DFS(char *senhaAlvo, char *tentativa, int index, int min, int max);
int senha_BFS(char *senhaAlvo, char *tentativa, int min, int max);
void reset_senha(char* senha, int value);

#endif

Overwriting search_pass.h


### Main:

In [ ]:
%%writefile main.c
#include "search_pass.h"
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <time.h>

int main(int argc, char *argv[]) {
    srand(time(NULL));

    char senha_target[TAMANHO_STRING + 1];
    char senha_encontrada[TAMANHO_STRING + 1];

    int min = argv[1][0];
    int max = argv[2][0];
    clock_t  time_inicio, time_fim;
    double tempo_execucao;

    gen_senha(senha_target, TAMANHO_STRING, min, max);
    printf("Senha alvo: %s\n", senha_target);

    reset_senha(senha_encontrada, min);
    time_inicio = clock();
    senha_DFS(senha_target, senha_encontrada, 0, min, max);
    time_fim = clock();
    tempo_execucao = (double)(time_fim - time_inicio) / CLOCKS_PER_SEC;

    printf("Senha encontrada: %s\n", senha_encontrada);
    printf("Número total de espaços explorados: %llu\n", iteracoes);
    printf("Tempo de execução: %lf segundos\n", tempo_execucao);

    reset_senha(senha_encontrada, min);
    senha_encontrada[TAMANHO_STRING] = '\0';
    time_inicio = clock();
    senha_BFS(senha_target, senha_encontrada, min, max);
    time_fim = clock();
    tempo_execucao = (double)(time_fim - time_inicio) / CLOCKS_PER_SEC;

    printf("Senha encontrada: %s\n", senha_encontrada);
    printf("Número total de espaços explorados: %llu\n", iteracoes);
    printf("Tempo de execução: %lf segundos\n", tempo_execucao);

    reset_senha(senha_encontrada, min);
    time_inicio = clock();
    gulosa_senha(senha_target, senha_encontrada, min, max);
    time_fim = clock();
    tempo_execucao = (double)(time_fim - time_inicio) / CLOCKS_PER_SEC;

    printf("Senha encontrada: %s\n", senha_encontrada);
    printf("Número total de espaços explorados: %llu\n", iteracoes);
    printf("Tempo de execução: %lf segundos\n", tempo_execucao);

    return 0;
}

Overwriting main.c


In [ ]:
!gcc -o procura_senhas main.c search_pass.c
!./procura_senhas A B

Senha alvo: BBBAAAAAAA
Senha encontrada: BBBAAAAAAA
Número total de espaços explorados: 1800
Tempo de execução: 0.000026 segundos
Senha encontrada: BBBAAAAAAA
Número total de espaços explorados: 1800
Tempo de execução: 0.000009 segundos
Senha encontrada: BBBAAAAAAA
Número total de espaços explorados: 30
Tempo de execução: 0.000002 segundos


### Script:

### MakeFile:

## Comparação e Resultados:


### Tempo de execução:

### Uso de memória:

### Nós explorados

# References:

In [ ]:
!gcc -g -o password_breaker password_breaker_busca_gulosa.c -o password_breaker_busca_gulosa
!valgrind --tool=massif ./password_breaker_busca_gulosa
!ms_print massif.out.xxxxx